In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample

from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [98]:
with open("census-bureau.columns", "r") as f:
    columns = [line.strip() for line in f.readlines() if line.strip()]

df = pd.read_csv("census-bureau.data", header=None, names=columns)

df.to_csv('df.csv', index=False)

df.shape

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 42 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   age                                         199523 non-null  int64  
 1   class of worker                             199523 non-null  object 
 2   detailed industry recode                    199523 non-null  int64  
 3   detailed occupation recode                  199523 non-null  int64  
 4   education                                   199523 non-null  object 
 5   wage per hour                               199523 non-null  int64  
 6   enroll in edu inst last wk                  199523 non-null  object 
 7   marital stat                                199523 non-null  object 
 8   major industry code                         199523 non-null  object 
 9   major occupation code                       199523 non-null  object 
 

In [99]:
df = df.replace('?', np.nan)
df = df.fillna("Missing")

In [100]:
#Correct the data type 
needtype_change = ["detailed industry recode", "detailed occupation recode", "own business or self employed", "veterans benefits", "year"]
df[needtype_change] = df[needtype_change].astype("object")

# Continuous variables: numeric types
continuous_vars = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Categorical variables: object types
categorical_vars = df.select_dtypes(include=["object"]).columns.tolist()

print("Continuous variables:", continuous_vars)
print("Categorical variables:", categorical_vars)

Continuous variables: ['age', 'wage per hour', 'capital gains', 'capital losses', 'dividends from stocks', 'weight', 'num persons worked for employer', 'weeks worked in year']
Categorical variables: ['class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc q

In [101]:
#processing data 
continuous_vars.remove('weight')
categorical_vars.remove('year')
categorical_vars.remove('label')

In [69]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_vars),
        ('cat', OneHotEncoder(drop='first'), categorical_vars)])

X_processed = preprocessor.fit_transform(df)

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

scores = []
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=3, max_iter=100, tol=1e-3)
    labels = kmeans.fit_predict(X_processed)
    score = silhouette_score(X_processed, labels)
    scores.append(score)
    print(f"k={k}: score={score:.3f}")

best_k = range(2, 8)[scores.index(max(scores))]
print(f"best k: {best_k}")

k=2: score=0.153
k=3: score=0.164
k=4: score=0.186
k=5: score=0.182
k=6: score=0.171
k=7: score=0.164
best k: 4


In [ ]:
#however, after invested the k, we would see that k=5 makes more sense and we can merge cluster 0 and 4 as they are all 

In [104]:
kmeans_final = KMeans(n_clusters=5, random_state=42)
clusters = kmeans_final.fit_predict(X_processed)

In [129]:
df['cluster'] = clusters
df['cluster'] = df['cluster'].replace(0, 1)

In [130]:
for i in range(1,5):
    mask = df['cluster'] == i
    people_count = df.loc[mask, 'weight'].sum()  
    data_points = mask.sum()
    print(f"Cluster {i}: {data_points} data point, represents {round(people_count)} people in national")

Cluster 1: 52197 data point, represents 92434646 people in national
Cluster 2: 40200 data point, represents 69121801 people in national
Cluster 3: 56536 data point, represents 95147356 people in national
Cluster 4: 50590 data point, represents 90542090 people in national


In [131]:
numeric_cols = df.select_dtypes(include=['number']).columns
numeric_cols = numeric_cols.drop('cluster')
cluster_means = df.groupby('cluster')[numeric_cols].mean().T
print(cluster_means.round(2))

cluster                                1        2        3        4
age                                38.22    60.96     8.61    38.54
wage per hour                     104.56     0.06     0.48   110.14
capital gains                     708.60   218.31     0.17   809.72
capital losses                     61.60    24.46     0.60    63.50
dividends from stocks             185.50   489.69     1.06   197.35
weight                           1770.88  1719.45  1682.95  1789.72
num persons worked for employer     3.59     0.13     0.06     3.84
weeks worked in year               43.75     0.82     0.27    45.32
w_norm                              0.00     0.00     0.00     0.00


In [132]:
df_new = df.copy()
df_new["age_band"] = pd.cut(df_new["age"],
        bins=[-1,10,17,24,34,44,54,64,200],
        labels=["<=10", "≤17","18–24","25–34","35–44","45–54","55–64","65+"])
df_new["weeks_band"] = pd.cut(df_new["weeks worked in year"],
        bins=[-0.1, 0, 26, 49, 52],
        labels=["didn’t work","~half year"," more than half year but not full year", "full year"])

In [133]:
categorical_vars_final = ["age_band", "weeks_band", "label"] + categorical_vars 

for col in categorical_vars_final:
    print(f"\n{col} (% in clusters):")
    pct_table = df_new.groupby('cluster')[col].value_counts(normalize=True).unstack(level=0) * 100
    print(pct_table.round(1))
    print("-" * 50)


age_band (% in clusters):
cluster      1     2     3     4
age_band                        
<=10       0.0   0.0  61.9   0.0
≤17        1.9   0.1  33.6   1.9
18–24     14.5   2.4   4.1  13.7
25–34     26.6   9.3   0.3  25.9
35–44     26.4  10.1   0.0  26.8
45–54     18.3   9.6   0.0  19.3
55–64      9.2  15.7   0.0   9.5
65+        3.1  52.8   0.0   2.9
--------------------------------------------------

weeks_band (% in clusters):
cluster                                    1     2     3     4
weeks_band                                                    
didn’t work                              4.0  94.1  97.4   1.9
~half year                              12.1   5.1   2.5  10.8
 more than half year but not full year  14.1   0.5   0.1  14.3
full year                               69.7   0.2   0.0  73.0
--------------------------------------------------

label (% in clusters):
cluster      1     2      3     4
label                            
- 50000.  89.4  98.3  100.0  87.8
50000+. 

In [134]:
#weighted within group
for col in categorical_vars_final:
    print(f"\n{col} (%):")
    
    df_temp = df_new.copy()
    df_temp['within_cluster_weight_pct'] = df_temp.groupby('cluster')['weight'].transform(lambda x: x / x.sum())
    
    pct_table = pd.crosstab(
        index=df_temp[col],
        columns=df_temp['cluster'], 
        values=df_temp['within_cluster_weight_pct'],
        aggfunc='sum'
    ) * 100
    
    print(pct_table.round(1))
    print("-" * 50)


age_band (%):
cluster      1     2     3     4
age_band                        
<=10       0.0   0.0  61.7   0.0
≤17        1.8   0.1  33.3   1.8
18–24     15.4   2.5   4.6  14.5
25–34     27.0   9.4   0.4  26.2
35–44     26.3  10.5   0.0  26.7
45–54     17.7   9.9   0.0  18.7
55–64      8.9  16.0   0.0   9.2
65+        3.0  51.6   0.0   2.8
--------------------------------------------------

weeks_band (%):
cluster                                    1     2     3     4
weeks_band                                                    
didn’t work                              4.4  94.3  97.4   1.9
~half year                              12.3   5.0   2.5  11.1
 more than half year but not full year  14.0   0.5   0.1  14.3
full year                               69.3   0.2   0.0  72.6
--------------------------------------------------

label (%):
cluster      1     2      3     4
label                            
- 50000.  89.3  98.3  100.0  87.7
50000+.   10.7   1.7    0.0  12.3
----------